In [ ]:
import sys
sys.path.append("../../")

In [ ]:
# an example with unnecessary object inheritance
class C(object):
    ...


# the above example can be simplified as this
class C:
    ...

In [ ]:
from fixit.common.base import CstLintRule


class NoInheritFromObjectRule(CstLintRule):
    def visit_ClassDef(self, node):
        ...

In [ ]:
import libcst as cst


class NoInheritFromObjectRule(CstLintRule):
    def visit_ClassDef(self, node: cst.ClassDef) -> None:
        ...

In [ ]:
def visit_If(self, node: cst.If) -> None:
    # called first
    ...


def visit_If_test(self, node: cst.If) -> None:
    # called after visit_If, but before we visit the test attribute
    # `leave_If_test` would be called next, followed by `leave_If`.
    if check_something(node.test):
        ...

In [ ]:
# check if any of the base classes of this class def is "object"
def visit_ClassDef(self, node: cst.ClassDef):
    has_object_base = any(
        isinstance(arg.value, cst.Name) and arg.value.value == "object"
        for arg in node.bases
    )

In [ ]:
import libcst.matchers as m


def visit_ClassDef(self, node: cst.ClassDef):
    has_object_base = m.matches(
        node, m.ClassDef(bases=[m.AtLeastN(n=1, matcher=m.Arg(value=m.Name("object")))])
    )

In [ ]:
class NoInheritFromObjectRule(CstLintRule):
    MESSAGE = "Inheriting from object is a no-op. 'class Foo:' is just fine =)"

    def visit_ClassDef(self, node: cst.ClassDef) -> None:
        new_bases = tuple(
            base for base in node.bases if not m.matches(base.value, m.Name("object"))
        )
        if tuple(node.bases) != new_bases:
            self.report(node)

In [ ]:
class NoInheritFromObjectRule(CstLintRule):
    MESSAGE = "Inheriting from object is a no-op. 'class Foo:' is just fine =)"

    def visit_ClassDef(self, node: cst.ClassDef) -> None:
        new_bases = tuple(
            base for base in node.bases if not m.matches(base.value, m.Name("object"))
        )

        if tuple(node.bases) != new_bases:
            # reconstruct classdef, removing parens if bases and keywords are empty
            new_classdef = node.with_changes(
                bases=new_bases,
                lpar=cst.MaybeSentinel.DEFAULT,
                rpar=cst.MaybeSentinel.DEFAULT,
            )

            # report warning and autofix
            self.report(node, replacement=new_classdef)

In [ ]:
class MyRule(CstLintRule):
    def should_skip_file(self):
        # Assert statements are okay for tests.
        # We could check the self.context.file_path object (see pathlib.Path), but
        # Context has a helper property for tests, since this is a common use-case
        return self.context.in_tests